In [1]:
import numpy as np # linear algebra
import pandas as pd# data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('data/train.csv')

**Заполнение NaN**

* HomePlanet

In [3]:
train['HomePlanet'].value_counts(normalize=True)

HomePlanet
Earth     0.541922
Europa    0.250942
Mars      0.207136
Name: proportion, dtype: float64

Можем заполнить HomePlanet модой - Earth  и заменить планеты на номера: Europa - 0, Earth - 1, Mars - 2

In [4]:
value = train['HomePlanet'].mode().iloc[0]
train['HomePlanet'].fillna(value, inplace=True)

In [5]:
homeplanet_mapping = {'Europa': 0, 'Earth': 1, 'Mars': 2}
train['HomePlanet'] = train['HomePlanet'].map(homeplanet_mapping).astype(int)

* CryoSleep

In [6]:
train['CryoSleep'].value_counts(normalize=True)

CryoSleep
False    0.641694
True     0.358306
Name: proportion, dtype: float64

Заменим наны значениями True и False случайным образом с полученными вероятностями и заменим False на 0, True на 1

In [7]:
cryosleep_probabilities = train['CryoSleep'].value_counts(normalize=True).to_dict()
train['CryoSleep'].fillna(pd.Series(np.random.choice(list(cryosleep_probabilities.keys()), p=list(cryosleep_probabilities.values()), size=train.shape[0])), inplace=True)

In [8]:
cryosleep_mapping = {train['CryoSleep'].unique()[0]: 0, train['CryoSleep'].unique()[1]: 1}
train['CryoSleep'] = train['CryoSleep'].map(cryosleep_mapping)

In [9]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,0,0,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,1,0,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,0,0,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,0,0,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,1,0,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


* Cabin

Создадим 3 доп. столбца: Deck, Num, Side

In [10]:
train[['Deck', 'Num', 'Side']] = train['Cabin'].str.split('/', expand=True)
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Num,Side
0,0001_01,0,0,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,B,0,P
1,0002_01,1,0,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,F,0,S
2,0003_01,0,0,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,A,0,S
3,0003_02,0,0,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,A,0,S
4,0004_01,1,0,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,F,1,S


In [11]:
print(train['Deck'].value_counts(normalize=True))
print(train['Side'].value_counts(normalize=True))

Deck
F    0.328938
G    0.301271
E    0.103132
B    0.091712
C    0.087944
D    0.056275
A    0.030139
T    0.000589
Name: proportion, dtype: float64
Side
S    0.504827
P    0.495173
Name: proportion, dtype: float64


Заполним Deck и Side аналогично CryoSleep

In [12]:
deck_probabilities = train['Deck'].value_counts(normalize=True).to_dict()
side_probabilities = train['Side'].value_counts(normalize=True).to_dict()
train['Deck'].fillna(pd.Series(np.random.choice(list(deck_probabilities.keys()), p=list(deck_probabilities.values()), size=train.shape[0])), inplace=True)
train['Side'].fillna(pd.Series(np.random.choice(list(side_probabilities.keys()), p=list(side_probabilities.values()), size=train.shape[0])), inplace=True)

In [13]:
side_mapping = {train['Side'].unique()[0]: 0, train['Side'].unique()[1]: 1}
train['Side'] = train['Side'].map(side_mapping).astype(int)
deck_unique_values = train['Deck'].unique()
deck_mapping = {value: index for index, value in enumerate(deck_unique_values)}
train['Deck'] = train['Deck'].map(deck_mapping).astype(int)

Num заполним медианой

In [14]:
train['Num'].fillna(train['Num'].median(), inplace=True)
train['Num'] = train['Num'].astype(int)

Всю информацию из Cabin мы получили, так что можно удалить этот столбец

In [15]:
train = train.drop('Cabin', axis=1)
train.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Num,Side
0,0001_01,0,0,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0,0,0
1,0002_01,1,0,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,1,0,1
2,0003_01,0,0,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,2,0,1
3,0003_02,0,0,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,2,0,1
4,0004_01,1,0,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,1,1,1


* Destination

In [16]:
train['Destination'].value_counts(normalize=True)

Destination
TRAPPIST-1e      0.694983
55 Cancri e      0.211491
PSO J318.5-22    0.093526
Name: proportion, dtype: float64

TRAPPIST-1e встречается гораздо чаще остальных, так что заполним им пропуски

In [17]:
value = train['Destination'].mode().iloc[0]
train['Destination'].fillna(value, inplace=True)

In [18]:
destination_mapping = {'TRAPPIST-1e': 0, 'PSO J318.5-22': 1, '55 Cancri e': 2}
train['Destination'] = train['Destination'].map(destination_mapping).astype(int)

Можем также создать столбец "Route", в который записать одно из 9 направлений:
1. Europa - TRAPPIST-1e
2. Europa - PSO J318.5-22
3. Europa - 55 Cancri e
4. Earth - TRAPPIST-1e
5. Earth - PSO J318.5-22
6. Earth - 55 Cancri e
7. Mars - TRAPPIST-1e
8. Mars - PSO J318.5-22
9. Mars - 55 Cancri e

In [19]:
def create_route(row):
    home_planet = row['HomePlanet']
    destination = row['Destination']
    route_number = home_planet * len(destination_mapping) + destination + 1
    return route_number

train['Route'] = train.apply(create_route, axis=1)

In [20]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Num,Side,Route
0,0001_01,0,0,0,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0,0,0,1
1,0002_01,1,0,0,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,1,0,1,4
2,0003_01,0,0,0,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,2,0,1,1
3,0003_02,0,0,0,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,2,0,1,1
4,0004_01,1,0,0,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,1,1,1,4


* Age

Заполним пропущенные возраста средним +- среднеквадратичное отклонение случайным образом

In [21]:
age_avg = train['Age'].mean()
age_std = train['Age'].std()
age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=train['Age'].isnull().sum())
train['Age'][np.isnan(train['Age'])] = age_null_random_list
train['Age'] = train['Age'].astype(int)
train.head()

C:\Users\draco\AppData\Local\Temp\ipykernel_1172\3906624399.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Age'][np.isnan(train['Age'])] = age_null_random_list


,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Num,Side,Route
0,0001_01,0,0,0,39,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0,0,0,1
1,0002_01,1,0,0,24,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,1,0,1,4
2,0003_01,0,0,0,58,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,2,0,1,1
3,0003_02,0,0,0,33,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,2,0,1,1
4,0004_01,1,0,0,16,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,1,1,1,4


Разделим возраст на 5 групп и заменим возраст на номер группы

In [22]:
train['AgeBand'] = pd.cut(train['Age'], 5)
train['Age'] = train['AgeBand'].cat.codes
train.drop('AgeBand', axis=1, inplace=True)
train.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Num,Side,Route
0,0001_01,0,0,0,2,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0,0,0,1
1,0002_01,1,0,0,1,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,1,0,1,4
2,0003_01,0,0,0,3,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,2,0,1,1
3,0003_02,0,0,0,2,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,2,0,1,1
4,0004_01,1,0,0,1,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,1,1,1,4


* VIP

In [23]:
train['VIP'].value_counts(normalize=True)

VIP
False    0.976561
True     0.023439
Name: proportion, dtype: float64

97% людей не использовали VIP, так что заполним пропуски значением False

In [24]:
value = train['VIP'].mode().iloc[0]
train['VIP'].fillna(value, inplace=True)
vip_mapping = {train['VIP'].unique()[0]: 0, train['VIP'].unique()[1]: 1}
train['VIP'] = train['VIP'].map(vip_mapping)

In [25]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Num,Side,Route
0,0001_01,0,0,0,2,0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0,0,0,1
1,0002_01,1,0,0,1,0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,1,0,1,4
2,0003_01,0,0,0,3,1,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,2,0,1,1
3,0003_02,0,0,0,2,0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,2,0,1,1
4,0004_01,1,0,0,1,0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,1,1,1,4


* RoomService, FoodCourt, ShoppingMall, Spa, VRDeck

Заполним их модами

In [26]:
for col in ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']:
    mode_value = train[col].mode().iloc[0]
    train[col].fillna(mode_value, inplace=True)

In [27]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Num,Side,Route
0,0001_01,0,0,0,2,0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0,0,0,1
1,0002_01,1,0,0,1,0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,1,0,1,4
2,0003_01,0,0,0,3,1,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,2,0,1,1
3,0003_02,0,0,0,2,0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,2,0,1,1
4,0004_01,1,0,0,1,0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,1,1,1,4


можно разбить их на группы и на основе группы разбить людей на классы

In [28]:
for col in ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']:
    bins = pd.cut(train[col], bins=[-np.inf, 0, 1, 2, 5, np.inf])
    train[col] = pd.cut(train[col], bins=[-np.inf, 0, 1, 2, 5, np.inf]).cat.codes

In [29]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Num,Side,Route
0,0001_01,0,0,0,2,0,0,0,0,0,0,Maham Ofracculy,False,0,0,0,1
1,0002_01,1,0,0,1,0,4,4,4,4,4,Juanna Vines,True,1,0,1,4
2,0003_01,0,0,0,3,1,4,4,0,4,4,Altark Susent,False,2,0,1,1
3,0003_02,0,0,0,2,0,0,4,4,4,4,Solam Susent,False,2,0,1,1
4,0004_01,1,0,0,1,0,4,4,4,4,2,Willy Santantines,True,1,1,1,4


Определим "соц уровень" на основе столбцов 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'

In [30]:
train['SocialLevel'] = train[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].mode(axis=1).iloc[:, 0]
train['SocialLevel'] = train['SocialLevel'].astype(int)
train.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Num,Side,Route,SocialLevel
0,0001_01,0,0,0,2,0,0,0,0,0,0,Maham Ofracculy,False,0,0,0,1,0
1,0002_01,1,0,0,1,0,4,4,4,4,4,Juanna Vines,True,1,0,1,4,4
2,0003_01,0,0,0,3,1,4,4,0,4,4,Altark Susent,False,2,0,1,1,4
3,0003_02,0,0,0,2,0,0,4,4,4,4,Solam Susent,False,2,0,1,1,4
4,0004_01,1,0,0,1,0,4,4,4,4,2,Willy Santantines,True,1,1,1,4,4


* Name

Удалим столбец Name

In [31]:
train = train.drop('Name', axis=1)
train.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Num,Side,Route,SocialLevel
0,0001_01,0,0,0,2,0,0,0,0,0,0,False,0,0,0,1,0
1,0002_01,1,0,0,1,0,4,4,4,4,4,True,1,0,1,4,4
2,0003_01,0,0,0,3,1,4,4,0,4,4,False,2,0,1,1,4
3,0003_02,0,0,0,2,0,0,4,4,4,4,False,2,0,1,1,4
4,0004_01,1,0,0,1,0,4,4,4,4,2,True,1,1,1,4,4


Заменим ID на группу пассажира, т.к. его номер в группе нас не интересует

In [32]:
train['GroupNumber'] = train['PassengerId'].str.split('_').str[0].astype(int)
train.drop('PassengerId', axis=1, inplace=True)

In [33]:
train.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Num,Side,Route,SocialLevel,GroupNumber
0,0,0,0,2,0,0,0,0,0,0,False,0,0,0,1,0,1
1,1,0,0,1,0,4,4,4,4,4,True,1,0,1,4,4,2
2,0,0,0,3,1,4,4,0,4,4,False,2,0,1,1,4,3
3,0,0,0,2,0,0,4,4,4,4,False,2,0,1,1,4,3
4,1,0,0,1,0,4,4,4,4,2,True,1,1,1,4,4,4


In [ ]:
columns_to_plot = train[train.columns.difference(['Transported'])]
num_columns = len(columns_to_plot)
fig, axes = plt.subplots(4, 4, figsize=(30, 20))
plt.subplots_adjust(hspace=0.5)

for i, column in enumerate(columns_to_plot):
    ax = axes[i // 4, i % 4]
    sns.barplot(
        x=train[column],  
        y=train['Transported'],  
        ax=ax,
        errorbar=None
    )
    ax.set_xlabel(column)
    ax.set_ylabel('Transported')  
    ax.set_title(f'{column} vs Transported')

plt.show()

In [ ]:
plt.figure(figsize=(15, 6))
sns.clustermap(train.corr("pearson"), vmin=-1.0, vmax=1.0, cmap="icefire", method="complete", annot=True)

Из матрицы корреляций видим следующие зависимости:
1. Transported достаточно сильно зависит от RoomService, FoodCourt, ShoppingMall, Spa, VRDeck и, в том числе, от SocialLevel. 
2. Transported также очень зависит от CryoSleep.
3. Вопреки ожиданиям, Transported имеет слабую корреляцию с возрастом, наличием VIP, палубой, маршрутом, номером каюты и номером группы; немного более высокую со стороной, пунктом прибытия и пунктом отправления.
4. RoomService, FoodCourt, ShoppingMall, Spa, VRDeck имеют сильную корреляцию между собой, которая увеличилась после разбиения значений на интервалы, и с SocialLevel, поэтому эти 5 признаков можно будет заменить одним. Также они коррелируют с CryoSleep

In [ ]:
from sklearn.model_selection import train_test_split

target = "Transported"
X = train.drop(target, axis=1)
y = train[target]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y, train_size=0.8, test_size=0.2, random_state=0)

In [ ]:
X_train.to_csv("data/X_train_preprocessed.csv", index=False)
y_train.to_csv("data/y_train_preprocessed.csv", index=False)
X_valid.to_csv("data/X_valid_preprocessed.csv", index=False)
y_valid.to_csv("data/y_valid_preprocessed.csv", index=False)

In [ ]:
with open('data/scores.txt', 'w') as file:
    file.truncate(0)